# Fine Tuning Existing LLM Models

In [ ]:
# Libraries
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install bitsandbytes
!pip install huggingface_hub
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import login
import torch

In [ ]:
# Dataset
dataset_name = '' # Add dataset to train on

In [ ]:
# Load Dataset
dataset = load_dataset(
    dataset_name,
    split="train"
)

In [ ]:
# Pre trained model
model_name = "" # Add model to fine tune

In [ ]:
# BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.float16, 
)

In [ ]:
# Loading Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

In [ ]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Lora Configuration
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# Training arguments
output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
# SFTTrainer
max_seq_length = 512
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(
            torch.float32
        )

In [ ]:
# Train
trainer.train()

In [ ]:
# Post training steps
model_to_save = trainer.model.module if hasattr(
    trainer.model,
    'module'
) else trainer.model

model_to_save.save_pretrained(
    "outputs"
)

lora_config = LoraConfig.from_pretrained(
    'outputs'
)

model = get_peft_model(
    model,
    lora_config
)

In [ ]:
# Text to predict on
text = "" # Add text to make prediction on

In [ ]:
# Device
device = "cuda:0"

In [ ]:
# Tokenize input
inputs = tokenizer(
    text,
    return_tensors="pt"
).to(
    device
)

In [ ]:
# Output
outputs = model.generate(
    **inputs,
    max_new_tokens=10
)

In [ ]:
# Print predicted output
print(
    tokenizer.decode(
        outputs[0],
        skip_special_tokens=True
    )
)